In [33]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/description.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt


# Importing libraries and datasets 

In [34]:
# Importing necessary libraries
import pandas as pd 
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

In [35]:
# Importing training dataset
df_train = pd.read_csv('/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt', 
                 sep=":::", header=None,names=["Id","Titl","Genre","Desc"],index_col=0)
df_train.head()

/tmp/ipykernel_32/2645902602.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_train = pd.read_csv('/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt',


,Titl,Genre,Desc
Id,,,
1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


In [36]:
# Importing testing dataset
df_test = pd.read_csv('/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt', 
                 sep=":::", header=None,names=["Titl","Genre","Desc"],index_col=0)
df_test.head()

/tmp/ipykernel_32/179991680.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_test = pd.read_csv('/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt',


,Titl,Genre,Desc
1,Edgar's Lunch (1998),thriller,"L.R. Brane loves his life - his car, his apar..."
2,La guerra de papá (1977),comedy,"Spain, March 1964: Quico is a very naughty ch..."
3,Off the Beaten Track (2010),documentary,One year in the life of Albin and his family ...
4,Meu Amigo Hindu (2015),drama,"His father has died, he hasn't spoken with hi..."
5,Er nu zhai (1955),drama,Before he was known internationally as a mart...


# Label Encoding

In [38]:
from sklearn.preprocessing import LabelEncoder
# Label encoding the output categories
le = LabelEncoder()
le2 = LabelEncoder()
le.fit(df_train.Genre)
le2.fit(df_test.Genre)
le.classes_

array([' action ', ' adult ', ' adventure ', ' animation ', ' biography ',
       ' comedy ', ' crime ', ' documentary ', ' drama ', ' family ',
       ' fantasy ', ' game-show ', ' history ', ' horror ', ' music ',
       ' musical ', ' mystery ', ' news ', ' reality-tv ', ' romance ',
       ' sci-fi ', ' short ', ' sport ', ' talk-show ', ' thriller ',
       ' war ', ' western '], dtype=object)

In [39]:
genres = le.transform(df_train.Genre)
df_train['Labeled Genre']=genres
genres2 = le2.transform(df_test.Genre)
df_test['Labeled Genre']=genres2

In [40]:
df_test.head()

,Titl,Genre,Desc,Labeled Genre
1,Edgar's Lunch (1998),thriller,"L.R. Brane loves his life - his car, his apar...",24
2,La guerra de papá (1977),comedy,"Spain, March 1964: Quico is a very naughty ch...",5
3,Off the Beaten Track (2010),documentary,One year in the life of Albin and his family ...,7
4,Meu Amigo Hindu (2015),drama,"His father has died, he hasn't spoken with hi...",8
5,Er nu zhai (1955),drama,Before he was known internationally as a mart...,8


# Splitting of Data

In [41]:
X_train=df_train.Desc
y_train=df_train['Labeled Genre']
X_test=df_test.Desc
y_test=df_test['Labeled Genre']

# TF-IDF

In [42]:
# Create a TF-IDF vectorizer to convert text messages to numerical features

feature = TfidfVectorizer(min_df=1, stop_words='english',lowercase=True)

# Convert the training and testing text messages into numerical features using TF-IDF

X_train_f = feature.fit_transform(X_train)
X_test_f = feature.transform(X_test)

# Modeling

## Logistic Regression

In [43]:
model = LogisticRegression(n_jobs=-1,max_iter=200)
model.fit(X_train_f,y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=200, n_jobs=-1)

In [44]:
# Model evaluation
prediction_on_training_data = model.predict(X_train_f)
accuracy_on_training_data = accuracy_score(y_train, prediction_on_training_data)

prediction_on_test_data = model.predict(X_test_f)
accuracy_on_test_data = accuracy_score(y_test, prediction_on_test_data)

# Print accuracy
print('Accuracy on training data: {} %'.format(accuracy_on_training_data * 100))
print('Accuracy on test data: {} %'.format(accuracy_on_test_data * 100))

Accuracy on training data: 72.03305419264396 %
Accuracy on test data: 58.808118081180815 %


## MultinomialNB 

In [45]:
from sklearn.naive_bayes import MultinomialNB
model2 = MultinomialNB()
model2.fit(X_train_f,y_train)

MultinomialNB()

In [46]:
# Model evaluation
prediction_on_training_data2 = model2.predict(X_train_f)
accuracy_on_training_data2 = accuracy_score(y_train, prediction_on_training_data2)

prediction_on_test_data2 = model2.predict(X_test_f)
accuracy_on_test_data2 = accuracy_score(y_test, prediction_on_test_data2)

# Print accuracy
print('Accuracy on training data: {} %'.format(accuracy_on_training_data2 * 100))
print('Accuracy on test data: {} %'.format(accuracy_on_test_data2 * 100))

Accuracy on training data: 47.12066993765448 %
Accuracy on test data: 44.474169741697416 %
